# Autogen Sanity Tests (In-Notebook)

This notebook runs quick, offline tests using `ipytest` and `pytest` to validate basic Python testing flow in VS Code.

In [2]:
# 1) Install and Verify Testing Tools
import sys, subprocess


def pip_install(packages):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *packages])
    except subprocess.CalledProcessError as e:
        print("pip install failed:", e)


pip_install(["ipytest", "pytest", "pytest-asyncio"])  # idempotent

import ipytest, pytest
from importlib.metadata import version, PackageNotFoundError


def pkg_version(name: str) -> str:
    try:
        return version(name)
    except PackageNotFoundError:
        return "unknown"


print("ipytest:", pkg_version("ipytest"))
print("pytest:", pkg_version("pytest"))

ipytest: 0.14.2
pytest: 8.4.2


In [4]:
# 2) Import and Configure ipytest
import ipytest, pytest
ipytest.autoconfig()

# Register the current notebook's namespace for test discovery
__name__ = "__main__"

In [5]:
# 3) Implement Code Under Test

def add(a, b):
    return a + b


def divide(a, b):
    if b == 0:
        raise ZeroDivisionError("division by zero")
    return a / b

# Example function that would be mocked
import requests

def fetch_url(url: str) -> str:
    r = requests.get(url, timeout=5)
    r.raise_for_status()
    return r.text[:100]  # only first 100 chars

In [6]:
# 4) Write Basic pytest Tests

def test_add():
    assert add(2, 3) == 5


def test_divide_basic():
    assert divide(10, 2) == 5


In [7]:
# 5) Run Tests in the Notebook
import ipytest
ipytest.run('-q')

..                                                                                           [100%].                                                                                           [100%]



<ExitCode.OK: 0>

In [8]:
# 6) Parametrized Tests
import pytest

@pytest.mark.parametrize("a,b,expected", [
    (1, 2, 3),
    (-1, 1, 0),
    (0, 0, 0),
])
def test_add_param(a, b, expected):
    assert add(a, b) == expected


@pytest.mark.parametrize("a,b,expected", [
    (10, 5, 2),
    (3, 2, 1.5),
])
def test_divide_param(a, b, expected):
    assert divide(a, b) == expected

In [9]:
# 7) Use Fixtures for Setup/Teardown
import tempfile, shutil
import pytest

@pytest.fixture
def temp_dir():
    d = tempfile.mkdtemp()
    try:
        yield d
    finally:
        shutil.rmtree(d)


def test_fixture_usage(temp_dir):
    import os
    p = os.path.join(temp_dir, "x.txt")
    with open(p, "w", encoding="utf-8") as f:
        f.write("ok")
    assert os.path.exists(p)

In [10]:
# 8) Mock External Dependencies
from unittest.mock import patch, MagicMock

@patch("requests.get")
def test_fetch_url_mocked(get_mock):
    m = MagicMock()
    m.status_code = 200
    m.text = "Hello World!"
    get_mock.return_value = m

    out = fetch_url("https://example.com")
    get_mock.assert_called_once()
    assert "Hello" in out

In [11]:
# 9) Test Exceptions and Edge Cases
import pytest

def test_divide_raises():
    with pytest.raises(ZeroDivisionError):
        divide(1, 0)


def test_boundary_values():
    assert add(0, 0) == 0
    assert divide(1, 1) == 1

In [12]:
# 10) Run pytest as a Shell Command
import os, json, textwrap, subprocess, sys, tempfile

code = textwrap.dedent('''
import pytest
from math import isclose

def add(a,b):
    return a+b

def test_add_pyfile():
    assert add(2,2) == 4
''')

with tempfile.TemporaryDirectory() as d:
    path = os.path.join(d, "t_test.py")
    with open(path, "w", encoding="utf-8") as f:
        f.write(code)
    try:
        subprocess.check_call([sys.executable, "-m", "pytest", "-q", path])
    except subprocess.CalledProcessError as e:
        print("pytest run failed:", e)

.                                                                        [100%]
1 passed in 0.01s
